In [86]:
import os
import numpy as np
import pandas as pd
import glob
import cv2
import matplotlib.pyplot as plt
import json

In [70]:
def createDataFrame(image_path, label_path):
    bbox = []
    image_path = list(glob.glob(os.path.join(image_path, "*")))
    label = []

    for i in list(glob.glob(os.path.join(label_path, "*"))):
        with open(i) as f:
            temp = f.read().split(" ")
            temp = list(map(lambda x: float(x), temp))
            label = temp[0]
            temp = temp[1:]
            bbox.append(temp)

    data = pd.DataFrame(data={"image_path": image_path, "bbox": bbox, "label": label})
    return data


train_data = createDataFrame(
    "car_dataset-master/train/images", "car_dataset-master/train/labels"
)
test_data = createDataFrame(
    "car_dataset-master/test/images", "car_dataset-master/test/labels"
)
valid_data = createDataFrame(
    "car_dataset-master/valid/images", "car_dataset-master/valid/labels"
)

In [57]:
cv2.imread("car_dataset-master/valid/images\\000005.jpg").shape

(64, 85, 3)

In [64]:
valid_data

,image_path,bbox,label
0,car_dataset-master/valid/images\000005.jpg,"[0.517647, 0.515625, 0.4, 0.265625]",0.0
1,car_dataset-master/valid/images\000009.jpg,"[0.476667, 0.693841, 0.376666, -0.110508000000...",0.0
2,car_dataset-master/valid/images\000011.jpg,"[0.5, 0.465278, 0.395833, 0.24305499999999997]",0.0
3,car_dataset-master/valid/images\000015.jpg,"[0.521277, 0.549296, 0.43617000000000006, 0.12...",0.0
4,car_dataset-master/valid/images\000016.jpg,"[0.520161, 0.467742, 0.32661300000000004, 0.12...",0.0
...,...,...,...
1613,car_dataset-master/valid/images\016152.jpg,"[0.508547, 0.511111, 0.3461540000000001, 0.377...",0.0
1614,car_dataset-master/valid/images\016153.jpg,"[0.51375, 0.523452, 0.30874999999999997, 0.313...",0.0
1615,car_dataset-master/valid/images\016162.jpg,"[0.503906, 0.538824, 0.4054690000000001, 0.254...",0.0
1616,car_dataset-master/valid/images\016163.jpg,"[0.494208, 0.53866, 0.29343700000000006, 0.074...",0.0


In [90]:
def yolo_to_coco_bbox(yolo_bbox, img_width, img_height):
    x_center, y_center, width, height = yolo_bbox
    
    xmin = (x_center - width / 2) * img_width
    ymin = (y_center - height / 2) * img_height
    width = width * img_width
    height = height * img_height

    return [xmin, ymin, width, height]

In [97]:
def createCOCODataset(dataframe):
    images = []
    categories = [{"id":0, "name":"car"}]
    annotations = []

    for i, (im_path, bbox, label) in enumerate(dataframe.values):
        img = cv2.imread(im_path)
        height = img.shape[0]
        width = img.shape[1]

        images.append({
            "width":width,
            "height":height,
            "id":i,
            "file_name":os.path.basename(im_path)
        })

        annotations.append({
            "id":i,
            "image_id":i,
            "category_id":int(label),
            "segmentation":[],
            "bbox":yolo_to_coco_bbox(bbox, width, height),
            "ignore":0,
            "iscrowd":0,
            "area":0
        })

        dataset_coco_format = {
            "categories": categories,
            "images": images,
            "annotations": annotations,
        }
        
    return dataset_coco_format



In [98]:
train_coco = createCOCODataset(train_data)
test_coco = createCOCODataset(test_data)
valid_coco = createCOCODataset(valid_data)

In [99]:
with open('train_coco.json', 'w', encoding='utf-8') as f:
    json.dump(train_coco, f, indent=4)
with open('test_coco.json', 'w', encoding='utf-8') as f:
    json.dump(test_coco, f, indent=4)
with open('valid_coco.json', 'w', encoding='utf-8') as f:
    json.dump(valid_coco, f, indent=4)